In [ ]:
%%writefile Sources/histogram-bug.cu
#include "dli.cuh"

constexpr float bin_width = 10;

__global__ void histogram_kernel(cuda::std::span<float> temperatures, 
                                 cuda::std::span<int> histogram)
{
  int cell = blockIdx.x * blockDim.x + threadIdx.x;
  int bin = static_cast<int>(temperatures[cell] / bin_width);
  int old_count = histogram[bin];
  int new_count = old_count + 1;
  histogram[bin] = new_count;
}

void histogram(cuda::std::span<float> temperatures, 
               cuda::std::span<int> histogram, 
               cudaStream_t stream)
{
  int block_size = 256;
  int grid_size = cuda::ceil_div(temperatures.size(), block_size);
  histogram_kernel<<<grid_size, block_size, 0, stream>>>(
    temperatures, histogram);
}

In [ ]:
import Sources.dli
Sources.dli.run("Sources/histogram-bug.cu")

In [ ]:
%%writefile Sources/atomic.cu
#include <cuda/std/span>
#include <cuda/std/atomic>
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>

__global__ void kernel(cuda::std::span<int> count)
{
    // Wrap data in atomic_ref
    cuda::std::atomic_ref<int> ref(count[0]);

    // Atomically increment the underlying value
    ref.fetch_add(1);
}

int main()
{
    thrust::device_vector<int> count(1);

    int threads_in_block = 256;
    int blocks_in_grid = 42;

    kernel<<<blocks_in_grid, threads_in_block>>>(
        cuda::std::span<int>{thrust::raw_pointer_cast(count.data()), 1});

    cudaDeviceSynchronize();

    thrust::host_vector<int> count_host = count;
    std::cout << "expected: " << threads_in_block * blocks_in_grid << std::endl;
    std::cout << "observed: " << count_host[0] << std::endl;
}

In [ ]:
!nvcc -arch=native Sources/atomic.cu -o /tmp/a.out -run